In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [2]:
working_url = 'https://www.president.gov.ua/news/speeches'

In [3]:
def get_speech_text_from_preview(preview):
    url = preview.find('a')['href']
    speech_page = BeautifulSoup(urllib.request.urlopen(url), 'html.parser')
    return speech_page.find('div', attrs={'itemprop':'articleBody'}).text.strip()

In [4]:
column_lambda_dictionary = {
    'headline': lambda preview: preview.find('h3').text,
    'url': lambda preview: preview.find('a')['href'],
    'date': lambda preview: preview.find('p', attrs={'class':'date'}).text.strip(),
    'speech_text': get_speech_text_from_preview
}

In [5]:
final_dict = { key: [] for key in column_lambda_dictionary.keys()}

In [6]:
final_dict

{'headline': [], 'url': [], 'date': [], 'speech_text': []}

In [7]:
is_last_page = False
disabled_class_name = 'disabled'

while not is_last_page:
    page = urllib.request.urlopen(working_url)
    
    page_soup = BeautifulSoup(page, 'html.parser')
    
    content = page_soup.find('div', attrs={'class': 'cat_list'})
    speech_previews = content.findAll('div', attrs={'class':'item_stat cat_stat'}) 
    
    for preview in speech_previews:
        for section in column_lambda_dictionary.keys():
            final_dict[section].append(column_lambda_dictionary[section](preview))
    
    next_page_object = page_soup.find('div', attrs={'class':'pagination'}).findAll('i')[-1].parent
    working_url = next_page_object['href']
    
    is_last_page = disabled_class_name in next_page_object.get("class")

In [8]:
data = pd.DataFrame.from_dict(final_dict)

In [9]:
data.head()

,headline,url,date,speech_text
0,\nЗвернення Президента з нагоди Дня українсько...,https://www.president.gov.ua/news/zvernennya-p...,14 березня 2019 року - 10:01,Шановні воїни!\nДорогі добровольці!\nЯк Презид...
1,\nВиступ Президента України на дебатах Генерал...,https://www.president.gov.ua/news/vistup-prezi...,20 лютого 2019 року - 18:18,"Пані Голово,\nВаші Високоповажності,\nПані і П..."
2,\nВиступ Президента під час спеціального засід...,https://www.president.gov.ua/news/vistup-prezi...,19 лютого 2019 року - 14:04,Великий друг України пане Дональд Туск!\nШанов...
3,\nВиступ Президента у Парламенті щодо закріпле...,https://www.president.gov.ua/news/vistup-prezi...,7 лютого 2019 року - 12:09,Високодостойний пане голово Верховної ради Укр...
4,\nВиступ Президента на інтронізації Предстояте...,https://www.president.gov.ua/news/vistup-prezi...,3 лютого 2019 року - 12:15,Ваше Блаженство!\nВисокопреосвященні Владики і...


In [10]:
FILENAME = 'speeches.csv'
FOLDER_PATH = '../data/'

In [11]:
data.to_csv(f'{FOLDER_PATH}{FILENAME}', encoding='utf-8')